In [2]:
!pip install lxml

In [3]:
!pip install bs4

In [4]:
from bs4 import BeautifulSoup
import requests

Для начала заходим в архив и генерируем ссылки на публикации каждого дня с начала 2011 года (появление архива) по конец 2020.

In [48]:
urls = []
for y in range(10):
    y = 2011 + y
    for m in range(1, 13):
        for d in range(1, 32):
            url = 'http://www.vpravda.ru/archive/' + str(y) + '-' + str(m) + '-' + str(d) + '/'
            urls.append(url)
# print(urls)


In [63]:
import re

Заходим по каждой из сгенерированных ссылок и собираем ссылки на конкретные статьи за этот день. Если статей в этот день не было, ссылка не добавляется.

In [88]:
def __get_page_content(url):
    session = requests.Session()
    return session.get(url).content.decode('ascii', errors='ignore')

texts_links = []
for u in range(len(urls)):
    file = __get_page_content(urls[u])
    soup = BeautifulSoup(file, 'lxml')
    tag2 = soup.select_one("aside", class_='sidebars')
    tag2.decompose()
    tag = soup.find('div', id='main')
    tag = tag.div
    tag = tag.div
    tag = tag.div
    if tag is not None:
        tags = tag.find_all('div', class_='field-content')
        for t in tags:
            t = t.a
            link = str(t)
            res = re.search(r'href="(.+)">', link)
            if res is not None:
                the_link = res.group(1)
                if the_link not in texts_links:
                    texts_links.append(the_link)
print(texts_links[:10])
#             link = link.find('a')
#             if link.has_attr('href'):
#                 texts_links.append(link['href'])
#                 print(link)

['/obshchestvo/artrit-mozhet-vliyat-na-prodolzhitelnost-zhizni-97976/', '/sport/mezhdu-nebom-i-zemley-107/', '/obshchestvo/otchet-ot-pervogo-lica-215/', '/obshchestvo/esli-ne-povernemsya-licom-k-vrachu-nikakaya-modernizaciya-ne-pomozhet-282/', '/obshchestvo/glavnaya-figura-potrebitel-281/', '/ekonomika/glavnaya-figura-potrebitel-307/', '/kultura/zhil-na-zemle-hudozhnik-429/', '/obshchestvo/nachalo-peremen-1634/', '/obshchestvo/ya-moroza-ne-boyus-s-golovoyu-okunus-1693/', '/obshchestvo/novyy-ustav-regiona-byl-prorabotan-i-prinyat-v-rekordnye-sroki-1933/']


Проходимся по списку всех статей, которые когда-либо были опубликованы на сайте Волгоградской правды, парсим код страницы и вытаскиваем из него автора, дату, заголовок, подзаголовок, текст. Добавляем источник и ссылку, сохраняем в нужном виде в один текстовый файл.

In [132]:
def __get_page_content(url):
    session = requests.Session()
    r = session.get(url)
    r.encoding = 'utf-8'
    return r

with open('full_texts.txt', 'w', encoding='utf-8') as ff:
    ff.write('')
    
for text in texts_links:
    actual_link = 'https://www.vpravda.ru/' + text
    page = __get_page_content(actual_link)
    with open('full_texts.txt', 'a', encoding='utf-8') as f:
        try:
            soup = BeautifulSoup(page.text, 'lxml', from_encoding='utf-8')
            tag = soup.find('div', id='main')
            if tag:
                tag = tag.find('div', id='content')
                title = tag.h1.text
                tag = tag.article
                date = tag.find('div', class_='field field-name-field-article-date field-type-date field-label-hidden')
                if date:
                    date = date.div
                    date = date.div
                    date = date.span.text
                author = tag.find('div', class_='field field-name-field-article-author field-type-text field-label-hidden')
                if author:
                    author = author.div
                    author = author.div.text

                text_main = tag.find('div', class_='field field-name-body field-type-text-with-summary field-label-hidden')
                if text_main:
                    text_main = text_main.div
                    text_undertitle = tag.find('div', class_='field field-name-field-article-lead field-type-text-long field-label-hidden')
                    if text_undertitle:
                        text_undertitle = text_undertitle.div
                        text_undertitle = str(text_undertitle.div.text)
                        text_main = str(text_main.div.text)
                        text = re.sub('\n', '', text)
                        text = text_undertitle + text_main
                    else:
                        text = text_main
                    f.write('=====\n' + actual_link + '\n' + 'Волгоградская правда' + '\n' + str(date)[:10] + '\n' + str(author) + '\n' + str(title) + '\n' + str(text) + '\n')
        except ConnectionResetError:
            pass
            

В итоге у нас получается файл full_texts.txt, в котором записаны подряд все статьи в хронологическом порядке. Этот файл мы будем использовать как корпус в lab2_graphs.ipynb.